In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from Login_info import user as user1, password as pass1
import time, math
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver',options=options)

In [3]:
time.sleep(3)
driver.get("https://www.linkedin.com")

time.sleep(1)

In [4]:
def Login(wd, user,password):
    # locate email form by_class_name
    wd.find_element_by_id('session_key').send_keys(user)
    time.sleep(1)
    
    # locate password form by_class_name
    wd.find_element_by_id('session_password').send_keys(password)
    
    # locate submit button by_class_name
    wd.find_element_by_class_name('sign-in-form__submit-button').click()
    time.sleep(1)

In [5]:
def Search_company(wd, company):
    # Search bar
    search = driver.find_element_by_class_name('search-global-typeahead__input.always-show-placeholder') 
    
    # Write company name
    search.send_keys(company_name)  
    
    # Wait
    time.sleep(2)    
    
    search.send_keys(Keys.ENTER)
    
    time.sleep(2)

In [17]:
def Posts_pagination(wd):
    time.sleep(2)
    
    # Get number of total posts
    wd.find_element_by_xpath(f'//button[contains(@aria-label,"Posts")]').click()
    amount_of_posts=int(wd.find_element_by_class_name('pb2.t-black--light.t-14').text.split(' ')[0])
    number_of_pages = math.ceil(amount_of_posts/10)
 
    # String to look
    href = 'https://www.linkedin.com/feed/update/urn:li:activity:'
    posts = {}
    
    # For each page
    for j in range(2):
        wd.get(f'https://www.linkedin.com/search/results/content/?keywords=Applaudo%20Studios&origin=SWITCH_SEARCH_VERTICAL&page={j+1}')
        print(f'Page: {j+1}')
        time.sleep(2)
        
        # Get posts url
        for i in wd.find_elements_by_xpath(f'//a[contains(@href,"{href}")]'):
            posts[i.get_attribute('href')] = 1
            
    return list(posts)
        

In [7]:
def get_post_reactions(wd, post_url):
    reactors_name = []
    reactors_href = []
    reactions_type = []
    
    reactions_window_btn = 'v-align-middle.social-details-social-counts__reactions-count'
    reactions_window_class_name = 'artdeco-modal__content.social-details-reactors-modal__content.ember-view'
    reactors_href_class_name = 'link-without-hover-state.ember-view'
    reactors_name_class_name = 'artdeco-entity-lockup__title.ember-view'
    reaction_class_name = 'reactions-icon.social-details-reactors-tab-body__icon.reactions-icon__consumption--small'
    
    # visiting post url
    wd.get(post_url)
    
    # opening reactions window
    reactions = wd.find_element_by_class_name(reactions_window_btn)
    n_reactions = int(reactions.text)
    reactions.click()
    
    # scroll down in reactions window
    href = 'https://www.linkedin.com/in/'                            
    panel = wd.find_element_by_class_name(reactions_window_class_name)
    for i in range(int(n_reactions/10)+1):
        wd.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', panel)
        time.sleep(2)
        
    for reactor in wd.find_elements_by_class_name(reactors_href_class_name):
        reactors_href.append(reactor.get_attribute('href'))
        
    time.sleep(2)
    
    for reactor in wd.find_elements_by_class_name(reactors_name_class_name):
        reactors_name.append(reactor.text)
        
    time.sleep(2)
        
    for reaction in wd.find_elements_by_class_name(reaction_class_name):
        reactions_type.append(reaction.get_attribute('alt'))
    
    time.sleep(2)
                
    reactions_dataframe = pd.DataFrame(columns=['username','profile_url','reaction'])
    reactions_dataframe['username'] = reactors_name
    reactions_dataframe['profile_url'] = reactors_href
    reactions_dataframe['reaction'] = reactions_type
    return reactions_dataframe

In [8]:
def get_load_more_btn(wd):
    try:
        return wd.find_element_by_xpath("//*[text()='Load more comments']")
#         return wd.find_element_by_class_name('comments-comments-list__load-more-comments-button.artdeco-button.artdeco-button--muted.artdeco-button--1.artdeco-button--tertiary.ember-view')
    except:
        return 'Element not found'

def get_post_comments(wd, post_url):
    commentors_name = []
    commentors_href=[]
    comments=[]
    
    commentors_name_class_name = 'comments-post-meta__name-text.hoverable-link-text.mr1'
    commentors_href_class_name = 'ember-view.comments-post-meta__profile-link.t-16.t-black.t-bold.tap-target'
    comments_class_name = 'comments-comment-item__main-content.feed-shared-main-content--comment.t-14.t-black.t-normal'
    
    #visiting post url
    wd.get(post_url)
    
    # loading all comments
    load_more_btn = get_load_more_btn(wd)
    
    while(load_more_btn!='Element not found'):
        load_more_btn.click()
        time.sleep(3)
        load_more_btn = get_load_more_btn(wd)
        print(load_more_btn)
        
    for commentor in wd.find_elements_by_class_name(commentors_name_class_name):
        commentors_name.append(commentor.text)
    
    time.sleep(2)
        
    for commentor in wd.find_elements_by_class_name(commentors_href_class_name):
        commentors_href.append(commentor.get_attribute('href'))
    
    time.sleep(2)
        
    for comment in wd.find_elements_by_class_name(comments_class_name):
        comments.append(comment.text)
        
                    
    comments_dataframe = pd.DataFrame(columns=['username','profile_url','comment'])
    comments_dataframe['username'] = commentors_name
    comments_dataframe['profile_url'] = commentors_href
    comments_dataframe['comment'] = comments
    
    return comments_dataframe

In [9]:
Login(driver,user1,pass1)

In [10]:
company_name = 'Applaudo Studios'  
Search_company(driver,company_name)

In [18]:
posts = Posts_pagination(driver)

Page: 1
Page: 2


In [19]:
post_reactions = get_post_reactions(driver, posts[0])
post_reactions.head()

username  \
0         Dennis Cañas Barrientos   
1      Evelyn Raquel Medina Cañas   
2  Jonathan David Fernandez Lopez   
3           Sharath Reddy Vutukur   
4                Sellinni Cordova   

                                         profile_url reaction  
0  https://www.linkedin.com/in/dennis-ca%C3%B1as-...     LIKE  
1  https://www.linkedin.com/in/raquel-medina-ca%C...     LIKE  
2  https://www.linkedin.com/in/jonathan-david-fer...     LIKE  
3  https://www.linkedin.com/in/sharath-reddy-vutu...     LIKE  
4  https://www.linkedin.com/in/sellinni-cordova-2...     LIKE

In [20]:
# NOMBRE DE LAS FUNCIONES
# make loading comments work
# driver.get(posts[2])
# 'https://www.linkedin.com/posts/anaromerolinkepower_equipopower-familiaempresarial-activity-6817819334484774912-B4R2/'

post_comments = get_post_comments(driver, posts[2])
post_comments.head()

Element not found


username  \
0  Francisco Jose Menendez Vasquez   
1                      Moris Ramos   
2                    Laura Escobar   
3                Frederic Orellana   
4     Nelson René Ramírez Orellana   

                                         profile_url  \
0  https://www.linkedin.com/in/francisco-jose-men...   
1  https://www.linkedin.com/in/moris-ramos-558ba755/   
2  https://www.linkedin.com/in/laura-escobar-b1a9...   
3  https://www.linkedin.com/in/frederic-orellana-...   
4  https://www.linkedin.com/in/nelson-ren%C3%A9-r...   

                            comment  
0                    Congrats Marce  
1         Más que orgulloso de vos!  
2              Felicidades Carlitos  
3  En hora bueno bro! Felicidades 👏  
4                Felicidades Marce!

In [22]:
post_reactions_and_comments = pd.concat([post_reactions,post_comments])
post_reactions_and_comments.to_csv('C:/Users/VanessaNunez/Downloads')

PermissionError: [Errno 13] Permission denied: 'C:/Users/VanessaNunez/Downloads'

In [ ]:
# reactions-icon social-details-reactors-tab-body__icon reactions-icon__consumption--small
# funcion que te devuelva nombre, href perfil, reaction
# given the post, returns name, href y reactions
# luego comentarios
# juntar comentarios y reacciones
# guardar en el csv

# NOMBRE DE LAS FUNCIONES

In [ ]:
for k in reactors.keys():
    print(k)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
driver.find_elements_by_class_name('social-details-reactors-tab-body')

In [ ]:
#driver.close()

In [ ]:
driver.window_handles

In [ ]:
driver.current_window_handle

In [ ]:
def Posts(wd):
    time.sleep(2)
    # Click in Posts
    posts_url = wd.current_url + "posts/"
    wd.get(posts_url)
    time.sleep(2)
    
    # Scroll till end of page
    inicial_yoffset = 0
    scrolled_yoffset = driver.execute_script("return window.pageYOffset;") + 1 # Inicial return is 0

    while scrolled_yoffset > inicial_yoffset:
        inicial_yoffset = driver.execute_script("return window.pageYOffset;")
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(1)
        scrolled_yoffset = driver.execute_script("return window.pageYOffset;")
        
    # String to look
    data_urn = 'urn:li:activity:'
    posts = {}
            
    # Get posts url
    for i in wd.find_elements_by_class_name("occludable-update.ember-view"):
        posts[i.get_attribute('href')] = 1

    # Click next page
    if j + 1 < n_pages:
        wd.find_elements_by_class_name('artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')[0].click()

    return posts
        